In [1]:
import selenium
from selenium import webdriver
import pandas as pd
import json
from fuzzywuzzy import process
from selenium.webdriver.common.by import By
from tqdm import tqdm
index=10
import time
from bs4 import BeautifulSoup
from urllib.request import Request, urlopen
from itertools import cycle
from urllib.request import Request, urlopen, ProxyHandler
from urllib.error import URLError, HTTPError

C:\Users\hp\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


Proxy Check

In [34]:
import threading 
import queue
import requests

q = queue.Queue()
valid_proxies = []

# Load proxies from file into the queue
with open(r"C:\Users\hp\Desktop\proxy2.txt", "r") as f:
    proxies = f.read().split("\n")
    for p in proxies:
        q.put(p)

def check_proxies():
    global q
    while not q.empty():
        proxy = q.get()
        try:
            res = requests.get("https://ipinfo.io/json",
                               proxies={"http": proxy,
                                        "https": proxy},
                               timeout=10)  # Add a timeout to avoid hanging indefinitely
            
            if res.status_code == 200:
                print(proxy)
                valid_proxies.append(proxy)
        except Exception as e:
            # Print any exceptions that occur during the request
            print(f"Error occurred with proxy {proxy}: {e}")
        finally:
            # Mark the task as done, whether successful or not
            q.task_done()

# Create and start threads
threads = []
for _ in range(10):
    thread = threading.Thread(target=check_proxies)
    thread.start()
    threads.append(thread)

# Wait for all threads to complete
for thread in threads:
    thread.join()

# After all threads finish, print the valid proxies
print("Valid Proxies:")
print(valid_proxies)

output_file = r"C:\Users\hp\Desktop\valid_proxies2.txt"
with open(output_file, "w") as f:
    for proxy in valid_proxies:
        f.write(proxy + "\n")


Error occurred with proxy 198.44.255.3:80: HTTPSConnectionPool(host='ipinfo.io', port=443): Max retries exceeded with url: /json (Caused by ProxyError('Cannot connect to proxy.', OSError('Tunnel connection failed: 400 Bad Request')))
Error occurred with proxy 5.135.83.214:80: HTTPSConnectionPool(host='ipinfo.io', port=443): Max retries exceeded with url: /json (Caused by ProxyError('Cannot connect to proxy.', OSError('Tunnel connection failed: 400 Bad Request')))
Error occurred with proxy 13.38.176.104:3128: HTTPSConnectionPool(host='ipinfo.io', port=443): Max retries exceeded with url: /json (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: self signed certificate (_ssl.c:1007)')))
Error occurred with proxy 3.21.101.158:3128: HTTPSConnectionPool(host='ipinfo.io', port=443): Max retries exceeded with url: /json (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: self sign

Scraping

In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.firefox.options import Options as FirefoxOptions
import random
import time
from urllib.parse import urlparse, parse_qs

# Load proxies from file
def load_proxies(filename):
    with open(filename) as f:
        proxies = f.read().splitlines()
    return proxies

# Function to set up Selenium WebDriver with a random proxy
def setup_driver_with_proxy(proxy_address):
    options = FirefoxOptions()
    options.add_argument('--proxy-server=%s' % proxy_address)
    driver = webdriver.Firefox(options=options)
    return driver

# Function to scrape product links from the current page
def scrape_product_links(driver):
    anchor_tags = driver.find_elements(By.CSS_SELECTOR, 'a.a-link-normal.s-underline-text.s-underline-link-text.s-link-style.a-text-normal')
    product_links = []
    for tag in anchor_tags:
        link_url = tag.get_attribute('href')
        if link_url:
            normalized_url = normalize_url(link_url)
            if normalized_url not in product_links:  # Check for duplicates
                product_links.append(normalized_url)
    return product_links

# Function to normalize URL by removing query parameters
def normalize_url(url):
    parsed_url = urlparse(url)
    # Remove query parameters and fragments
    return parsed_url.scheme + '://' + parsed_url.netloc + parsed_url.path

# URL to scrape
url = "https://www.amazon.com"

# Load proxies from file
proxies = load_proxies(r"C:\Users\hp\Desktop\valid_proxies2.txt")

# Set up Selenium WebDriver
driver = setup_driver_with_proxy(random.choice(proxies))

# List to store unique product links in order
unique_product_links = []

# Scrape product links from 10 pages
for _ in range(25):
    # Load the page
    driver.get(url)
    
    # Scrape product links from the current page and add to the list
    unique_product_links.extend(scrape_product_links(driver))
    
    # Find the next page link
    next_page_link = driver.find_element(By.CSS_SELECTOR, 'a.s-pagination-item.s-pagination-next.s-pagination-button.s-pagination-separator')
    
    # Get the URL of the next page
    url = next_page_link.get_attribute('href')
    
    # Wait for a moment before navigating to the next page
    time.sleep(2)

# Print all unique product links
for link in unique_product_links:
    print(link)

# Close the WebDriver session
#driver.quit()


NoSuchElementException: Message: Unable to locate element: a.s-pagination-item.s-pagination-next.s-pagination-button.s-pagination-separator; For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
RemoteError@chrome://remote/content/shared/RemoteError.sys.mjs:8:8
WebDriverError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:191:5
NoSuchElementError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:509:5
dom.find/</<@chrome://remote/content/shared/DOM.sys.mjs:136:16


In [3]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.firefox.options import Options as FirefoxOptions
import random
import time
from urllib.parse import urlparse, parse_qs
from bs4 import BeautifulSoup



# Load proxies from file
def load_proxies(filename):
    with open(filename) as f:
        proxies = f.read().splitlines()
    return proxies

# Function to set up Selenium WebDriver with a random proxy
def setup_driver_with_proxy(proxy_address):
    options = FirefoxOptions()
    options.add_argument('--proxy-server=%s' % proxy_address)
    driver = webdriver.Firefox(options=options)
    return driver



# URL to scrape
url = "https://www.amazon.com/s?i=specialty-aps&bbn=16225019011&rh=n%3A7141123011%2Cn%3A16225019011%2Cn%3A679255011&ref=nav_em__nav_desktop_sa_intl_shoes_0_2_13_3"

# Load proxies from file
proxies = load_proxies(r"C:\Users\hp\Desktop\valid_proxies2.txt")

with open(r"C:\Users\hp\Desktop\ShoesUrlMore.txt") as f:
    shoes = f.read().splitlines()
    
    

# Set up Selenium WebDriver
driver = setup_driver_with_proxy(random.choice(proxies))

# List to store unique product links in order
ProductName=[]
Color=[]
Descrition=[]
About=[]
SubCategory=[]
Price=[]
Brand=[]


c=[]
# Scrape product links from 10 pages
#for i in shoes:
# Load the page
driver.get(shoes[0])

page_source = driver.page_source



pname = driver.find_element(By.ID, "title")



print(pname.text)

try:
    price= driver.find_element(By.CSS_SELECTOR, ".a-price")

    
    price=price.text.split('\n')[0]

except:
    price=""


print(price)
 
 

try: 

    elements = driver.find_elements(By.CLASS_NAME, "imgSwatch")
    if elements:

        for element in elements:
            color = element.get_attribute("alt")
            c.append(color)
    else:
        color=['No Color']
    
except:
       
    pass
        


soup = BeautifulSoup(page_source, 'html.parser')


  
about = soup.find_all("ul", class_="a-unordered-list a-vertical a-spacing-small")

for about_element in about:
    aboutUs = about_element.find_all("span", class_="a-list-item a-size-base a-color-base")
    for i in aboutUs:
        print(i.text)  
        

desc = soup.findAll("div", class_="a-row feature")


desc_elements = [desc_div.findAll("p") for desc_div in desc]

for element_list in desc_elements:
    for element in element_list:
        print(element.text)
        
        
def clean_text(text):
    # Remove Unicode left-to-right mark and strip leading/trailing whitespace/newlines
    return text.replace('\u200e', '').strip()

# Assuming 'soup' is your BeautifulSoup object
divs = soup.findAll("div", class_="a-section feature detail-bullets-wrapper bucket")

asin_data = []
manufacturer_data = []

for div in divs:
    spans = div.find_all("span")
    for span in spans:
        text = span.text
        if 'ASIN' in text:
            parts = text.split(':')
            if len(parts) > 1:
                asin = clean_text(parts[1])
                if asin:  # Ensure asin is not empty
                    asin_data.append(asin)
        elif 'Manufacturer' in text:
            parts = text.split(':')
            if len(parts) > 1:
                manufacturer = clean_text(parts[1])
                if manufacturer:  # Ensure manufacturer is not empty
                    manufacturer_data.append(manufacturer)

print(asin_data)
print(manufacturer_data)


reviews = soup.find_all('div', class_='a-expander-content reviewText review-text-content a-expander-partial-collapse-content')

for review in reviews:
    spans = review.find_all("span")  # Use find_all directly on the review container
    for span in spans:
        print(span.text)
        
stars = soup.find_all("span", class_="a-icon-alt")

for j in stars:
    if "star" in j.text:
            print(j.text)


sub = soup.find_all("a", class_="a-link-normal a-color-tertiary")

print(sub)

k=0
for i in sub:
    if k!=3:
        
        pass
    else:
        print(i.text)
    k+=1
# Wait for a moment before navigating to the next page
time.sleep(2)



# Close the WebDriver session
#driver.quit()


[<a class="a-link-normal a-color-tertiary" href="/amazon-fashion/b/ref=dp_bc_aui_C_1/138-5068103-0429128?ie=UTF8&amp;node=7141123011">
                Clothing, Shoes &amp; Jewelry
            </a>, <a class="a-link-normal a-color-tertiary" href="/Mens-Fashion/b/ref=dp_bc_aui_C_2/138-5068103-0429128?ie=UTF8&amp;node=7147441011">
                Men
            </a>, <a class="a-link-normal a-color-tertiary" href="/Mens-Shoes/b/ref=dp_bc_aui_C_3/138-5068103-0429128?ie=UTF8&amp;node=679255011">
                Shoes
            </a>, <a class="a-link-normal a-color-tertiary" href="/Mens-Sneakers/b/ref=dp_bc_aui_C_4/138-5068103-0429128?ie=UTF8&amp;node=6127770011">
                Athletic
            </a>, <a class="a-link-normal a-color-tertiary" href="/Running-Mens-Shoes/b/ref=dp_bc_aui_C_5/138-5068103-0429128?ie=UTF8&amp;node=679286011">
                Running
            </a>, <a class="a-link-normal a-color-tertiary" href="/Mens-Road-Running-Shoes/b/ref=dp_bc_aui_C_6/138-5068103-04

In [142]:
with open(r"C:\Users\hp\Desktop\ShoesUrlMore.txt", 'w') as f:
        for url in unique_product_links:
            f.write(url + '\n')

In [60]:
import json
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.firefox.options import Options as FirefoxOptions
import random
import time
from urllib.parse import urlparse, parse_qs
from bs4 import BeautifulSoup
from amazoncaptcha import AmazonCaptcha
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC




# Load proxies from file
def load_proxies(filename):
    with open(filename) as f:
        proxies = f.read().splitlines()
    return proxies

# Function to set up Selenium WebDriver with a random proxy
def setup_driver_with_proxy(proxy_address):
    options = FirefoxOptions()
    options.add_argument('--proxy-server=%s' % proxy_address)
    driver = webdriver.Firefox(options=options)
    return driver

def clean_text(text):
    # Remove Unicode left-to-right mark and strip leading/trailing whitespace/newlines
    return text.replace('\u200e', '').strip()

def bypass_captcha(driver):
    while True:
        WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.XPATH, "//*[@class='a-row a-text-center']//img")))
        
        captcha_imgs = driver.find_elements(By.XPATH, "//*[@class='a-row a-text-center']//img")
        captcha_vals = []
        for captcha_img in captcha_imgs:
            captcha_src = captcha_img.get_attribute('src')
            captcha = AmazonCaptcha.fromlink(captcha_src)
            captcha_val = AmazonCaptcha.solve(captcha)
            captcha_vals.append(captcha_val)
        
        input_fields = driver.find_elements(By.ID, "captchacharacters")
        for input_field, captcha_val in zip(input_fields, captcha_vals):
            input_field.send_keys(captcha_val)
        
        button = driver.find_element(By.CLASS_NAME, "a-button-text")
        button.click()
        
        WebDriverWait(driver, 10).until_not(EC.url_contains("captcha"))  # Wait until captcha is bypassed
        
        # Check if there are more captchas
        if 'captcha' not in driver.page_source:
            break
            
    return driver.page_source



# URL to scrape
url = "https://www.amazon.com/s?i=specialty-aps&bbn=16225019011&rh=n%3A7141123011%2Cn%3A16225019011%2Cn%3A679255011&ref=nav_em__nav_desktop_sa_intl_shoes_0_2_13_3"

# Load proxies from file
proxies = load_proxies(r"D:\Downloads\valid_proxy.txt")

with open(r"C:\Users\hp\Desktop\ShoesUrlMore.txt") as f:
    shoes = f.read().splitlines()
    
shoes = shoes[690:]
    

# Dictionary to hold product data
products_data = []

driver = setup_driver_with_proxy(random.choice(proxies))

driver.get("https://www.amazon.com")
# Loop through each shoe URL
for shoe_url in shoes:
    # Set up Selenium WebDriver
   
    
    driver.get(shoe_url)
    
    if 'captcha' in driver.page_source:
        bypass_captcha(driver)
   
    # Dictionary to hold data for the current product
    product_data = {}
    
    # Scrape product data
    product_data['url'] = shoe_url
    product_data['name'] = driver.find_element(By.ID, "title").text.strip()
    
    try:
        price_element = driver.find_element(By.CSS_SELECTOR, ".a-price")
        product_data['price'] = price_element.text.split('\n')[0]
    except:
        product_data['price'] = ""
    
    try:
        color_elements = driver.find_elements(By.CLASS_NAME, "imgSwatch")
        product_data['colors'] = [element.get_attribute("alt") for element in color_elements]
    except:
        product_data['colors'] = ['No Color']
    
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    
    about_elements = soup.find_all("ul", class_="a-unordered-list a-vertical a-spacing-small")
    product_data['about'] = [item.text.strip() for about_element in about_elements for item in about_element.find_all("span", class_="a-list-item a-size-base a-color-base")]
    
    
    description_elements = soup.find_all("div", class_="a-row feature")
    product_data['description'] = []
    for desc_div in description_elements:
        element_list = desc_div.find_all("p")
        for element in element_list:
            product_data['description'].append(element.text.strip())
    
    
    product_data['asin'] = []
    product_data['manufacturer'] = []
    divs = soup.findAll("div", class_="a-section feature detail-bullets-wrapper bucket")

    asin_data = []
    manufacturer_data = []

    for div in divs:
        spans = div.find_all("span")
        for span in spans:
            text = span.text
            if 'ASIN' in text:
                parts = text.split(':')
                if len(parts) > 1:
                    asin = clean_text(parts[1])
                    if asin:  # Ensure asin is not empty
                        product_data['asin'].append(asin)
            elif 'Manufacturer' in text:
                parts = text.split(':')
                if len(parts) > 1:
                    manufacturer = clean_text(parts[1])
                    if manufacturer:  # Ensure manufacturer is not empty
                        product_data['manufacturer'].append(manufacturer)
    
    reviews = soup.find_all('div', class_='a-expander-content reviewText review-text-content a-expander-partial-collapse-content')
    product_data['reviews'] = [review.text.strip() for review in reviews]
    
    stars = soup.find_all("span", class_="a-icon-alt")
    product_data['stars'] = [star.text.strip() for star in stars if "star" in star.text]
    
    sub_categories = soup.find_all("a", class_="a-link-normal a-color-tertiary")
    product_data['sub_category'] = ""
    for idx, sub_category in enumerate(sub_categories):
        if idx == 3:
            product_data['sub_category'] = sub_category.text.strip()
            break
    
    product_data["Gender"]="Male"
    
    product_data["Category"]="Shoes"
    # Append product data to the list
    products_data.append(product_data)
    
    # Close the WebDriver session
    #driver.quit()

# # Write data to JSON file
# with open('products_data.json', 'w') as json_file:
#     json.dump(products_data, json_file, indent=4)


NoSuchElementException: Message: Unable to locate element: [id="title"]; For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
RemoteError@chrome://remote/content/shared/RemoteError.sys.mjs:8:8
WebDriverError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:191:5
NoSuchElementError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:509:5
dom.find/</<@chrome://remote/content/shared/DOM.sys.mjs:136:16


In [61]:
len(products_data)


44

In [62]:
import json

# Function to append data to a JSON file
def append_to_json_file(filename, data):
    with open(filename, 'a+') as json_file:
        # Move the file pointer to the beginning to ensure that data is appended at the end
        json_file.seek(0)
        try:
            # Load existing data if any
            existing_data = json.load(json_file)
        except json.decoder.JSONDecodeError:
            # If file is empty, initialize with an empty list
            existing_data = []
        
        # Append new data to existing data
        existing_data.append(data)
        
        print(len(existing_data))
        # Move the file pointer to the beginning to overwrite existing content
        json_file.seek(0)
        
        # Write the updated data to the file
        json.dump(existing_data, json_file, indent=4)
        # Truncate the file to the current position (removes any remaining content)
        json_file.truncate()

# Example usage:


# Append the new data to the JSON file
append_to_json_file('products_data.json', products_data)


1


In [54]:
import json

# Load JSON data line by line
data = {}
with open('products_data.json', 'r') as file:
    for line in file:
        try:
            data.append(json.loads(line))
        except json.JSONDecodeError:
            pass  # Skip lines that don't contain valid JSON

# Count the number of products
num_products = len(data)

print("Number of products:", num_products)


AttributeError: 'dict' object has no attribute 'append'

In [55]:
data

{}